# Digit Recognizer

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch, torchvision
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
data = pd.read_csv('dataset/test.csv')
data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data = np.array(data)
# data = data.reshape(28000, 28, 28)

# data.shape

(28000, 28, 28)

In [15]:
data[27999]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  72,
        59,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

### Let's plan

For the neural network, let's do it as:
784 -> 128 -> 64 -> 10

784 are the pixels of the image, 128 and 64 are the number of neurons in the hidden layers and 10 is the number of classes. 

We will use the ReLU activation function for the hidden layers and the softmax activation function for the output layer.

i.e., ReLU for 128 and 64 neurons and softmax for 10 neurons.

ReLU formula is:

$relu(x) = max(0, x)$

Softmax formula is:

$softmax(x) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}$

But how will we verify the prediction with the real MNIST dataset? We will use the cross-entropy loss function. And? We will use the gradient descent optimizer to minimize the loss function. How will we show the output of the model? We will use the accuracy metric. 

Format of the result should be like this:

```
ImageId,Label
1,0
2,0
3,0
etc.
```